In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import os
import sys
import warnings
import numpy

def warn(*args, **kwargs):
    pass

warnings.warn = warn
warnings.simplefilter(action='ignore', category=FutureWarning)
numpy.seterr(divide = 'ignore') 

sys.path.append(os.path.dirname(os.path.abspath('')))

In [2]:
import ccxt

from tensorforce.agents import Agent
from tensorforce.environments import Environment

from tensortrade.environments import TradingEnvironment
from tensortrade.exchanges.simulated import FBMExchange
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features.stationarity import FractionalDifference
from tensortrade.features import FeaturePipeline
from tensortrade.rewards import SimpleProfitStrategy
from tensortrade.actions import DiscreteActionStrategy
from tensortrade.strategies import TensorforceTradingStrategy

normalize = MinMaxNormalizer(inplace=True)
difference = FractionalDifference(difference_order=0.6,
                                  inplace=True,
                                  all_column_names=["open", "high", "low", "close", "volume"])
feature_pipeline = FeaturePipeline(steps=[normalize, difference])

reward_strategy = SimpleProfitStrategy()
action_strategy = DiscreteActionStrategy(n_actions=20, instrument_symbol='ETH/BTC')

exchange = FBMExchange(base_instrument='BTC',
                       timeframe='1h',
                       should_pretransform_obs=True)

environment = TradingEnvironment(exchange=exchange,
                                 action_strategy=action_strategy,
                                 reward_strategy=reward_strategy,
                                 feature_pipeline=feature_pipeline)

agent_spec = {
    "type": "ppo_agent",
    "step_optimizer": {
        "type": "adam",
        "learning_rate": 1e-4
    },
    "discount": 0.99,
    "likelihood_ratio_clipping": 0.2,
}

network_spec = [
    dict(type='dense', size=64, activation="tanh"),
    dict(type='dense', size=32, activation="tanh")
]

strategy = TensorforceTradingStrategy(environment=environment, agent_spec=agent_spec, network_spec=network_spec)

performance = strategy.run(episodes=1, testing=True)

performance[-5:]

  0%|          | 0/1 [00:00<?, ?it/s]/Users/adam/Desktop/Hedger/tensortrade/tensortrade/rewards/simple_profit_strategy.py:51: RuntimeWarning: divide by zero encountered in log10
  return profit_sign * (1 + (5 ** np.log10(abs(profit))))
100%|██████████| 1/1 [00:09<00:00,  9.60s/it]

Finished running strategy.
Total episodes: 1 (1666 timesteps).
Average reward: 1099.9123929937914.


,balance,net_worth
1661,9651.275254,9900.965902
1662,9651.275254,9882.422803
1663,9588.175571,9887.980984
1664,9588.175571,9659.368193
1665,9540.877239,9757.674916
